# **자연어 4일 - Embadding, Search Engine**
- Document Representation : **[Source Code](http://bitly.kr/XKcjfe)**
- **[스탠포드 공대 IR 수업자료](https://nlp.stanford.edu/IR-book/newslides.html)** 
- **[버지니아 공대 NLP 수업자료](http://www.cs.virginia.edu/~hw5x/Course/IR2015/_site/lectures/)**

# **Preview**
1. **Konlpy 의 모듈** 중, 명사구분/ 형용사 구분 등 **필요한 성능에 따라** 다양한 모듈의 적용
1. Komoran 을 활용시 **java.lang.OutOfMemoryError: GC overhead limit exceeded** 오류시 주의할 것
1. 때문에 여러가지 이유로 **Mecab** 을 추천 합니다.

```python
# java.lang.OutOfMemoryError 오류시 Java 메모리 할당 추가하기
# 참고 사이트 : https://github.com/konlpy/konlpy/issues/93
import os
os.environ['JAVA_OPTS'] = '-Xmx4096M' # -Xmx1024M
```

In [1]:
# 어절 생성하는 함수 (N-Gram)
def eojeol(sent, num=2): # num > 2
    ngram = []
    token = sent.split() # Token 의 생성 (각자 다르게 적용)
    for i in range(len(token) - (num-1)):
        ngram.append(tuple(token[i: i+num]))
    return ngram

from konlpy.tag import Mecab
sentence = "아버지가방에들어가다."
print(Mecab().tagset)
print(Mecab().pos(sentence))
[_  for _ in Mecab().pos(sentence) if _[1] not in ["EF", "SF"]]

{'EC': '연결 어미', 'EF': '종결 어미', 'EP': '선어말어미', 'ETM': '관형형 전성 어미', 'ETN': '명사형 전성 어미', 'IC': '감탄사', 'JC': '접속 조사', 'JKB': '부사격 조사', 'JKC': '보격 조사', 'JKG': '관형격 조사', 'JKO': '목적격 조사', 'JKQ': '인용격 조사', 'JKS': '주격 조사', 'JKV': '호격 조사', 'JX': '보조사', 'MAG': '일반 부사', 'MAJ': '접속 부사', 'MM': '관형사', 'NNB': '의존 명사', 'NNBC': '단위를 나타내는 명사', 'NNG': '일반 명사', 'NNP': '고유 명사', 'NP': '대명사', 'NR': '수사', 'SC': '구분자 , · / :', 'SE': '줄임표 …', 'SF': '마침표, 물음표, 느낌표', 'SH': '한자', 'SL': '외국어', 'SN': '숫자', 'SSC': '닫는 괄호 ), ]', 'SSO': '여는 괄호 (, [', 'SY': '기타 기호', 'VA': '형용사', 'VCN': '부정 지정사', 'VCP': '긍정 지정사', 'VV': '동사', 'VX': '보조 용언', 'XPN': '체언 접두사', 'XR': '어근', 'XSA': '형용사 파생 접미사', 'XSN': '명사파생 접미사', 'XSV': '동사 파생 접미사'}
[('아버지', 'NNG'), ('가', 'JKS'), ('방', 'NNG'), ('에', 'JKB'), ('들어가', 'VV'), ('다', 'EF'), ('.', 'SF')]


[('아버지', 'NNG'), ('가', 'JKS'), ('방', 'NNG'), ('에', 'JKB'), ('들어가', 'VV')]

# **Word Token Embadding**
## **1. 자료 불러오기**
- Token String 을 **Index** 로 사용하면 자료가 커진다
- **Token Index** 를 외래키로 사용하여 **DataBase** 크기를 줄인다

```python
tokens = word_tokenize("\n".join(collection))
bigram = eojeol("\n".join(collection))
pos    = Okt().pos("\n".join(collection))
len(tokens + bigram + Voca), len(set(tokens + bigram + Voca))
```

In [2]:
# 정규식에서 숫자는 : String Range 값을 특정 합니다
import os, re
fileFolder = "./News/" 
fileList   = [fileFolder + _  for _ in os.listdir(fileFolder) 
               if re.match("\d{10}.txt", _) ]

# 네이버 뉴스 수집자료를 collection 로 묶는다
collection = []
for _ in fileList:
    with open(_) as f:
        collection.append(f.read())

# 문서 검색력을 높이기 위해 Token, Bigram, konlpy Tag 로 세분화 한다
from konlpy.tag import Mecab
from nltk.tokenize import word_tokenize
Vocabulary = Mecab().nouns("\n".join(collection))
bigram     = eojeol(" ".join(Vocabulary))

print("Collection Pages : {}".format(len(collection)))
len(bigram + Vocabulary), len(set(bigram + Vocabulary))

Collection Pages : 103


(66151, 29259)

## **2. Token 의 생성**
- Token String 을 **Index** 로 사용하면 자료가 커진다
- **Token Index** 를 외래키로 사용하여 **DataBase** 크기를 줄인다

```python
docRepresentation = {}
for token in word_tokenize(collection[0]):
    docRepresentation[Vocabulary.index(token)] = 1

docRepresentation
{114: 1, 9: 1, 92: 1, 111: 1, 22: 1, 72: 1 ...}
```

In [3]:
# Token List를 바탕으로 Index 값 추출
docIndex   = 20
Vocabulary = list(set(Mecab().nouns(collection[docIndex])))
print(Vocabulary[:15])
Vocabulary.index("국방")

['일', '부상', '미사일', '전열', '결속', '독', '전투기', '실질', '분석', '방문', '추가', '무단', '대북', '북한', '중국']


153

In [4]:
# Token 의 Index 로 변환된 자료형을 생성합니다
docRepresentation = {}
for token in Mecab().nouns(collection[docIndex]):
    docRepresentation[Vocabulary.index(token)] = 1
print(len(docRepresentation))
docRepresentation.values()

274


dict_values([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [5]:
vector = []
for i in range(len(Vocabulary)):
    if i in docRepresentation:
        vector.append(docRepresentation[i])
    else:
        vector.append(0)
vector[:20]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

# **Search Engine Index 만들기**
**Word Token Indexing** , 인덱싱 자료를 활용한 **검색 Tree** 만들기
1. **AD HOC 방식** : 고정된 데이터를 대상으로 **사용자의 요구에 따라** 쿼리가 변경
2. **Filtering (추천시스템)** : 사용자의 요구는 동일하지만 **새로운 문서 및 문서의 가중치가 자동으로 변경**

## **1 DTM 모델 만들기**
**Word Token Indexing** 를 활용한 **Document Term Matrix** 만들기
1. 장점은 쉽게 만들수 있다
1. 단점으로는 **전체 Token을 확인하는 만큼** 검색 비용이 많이 든다

```json
DTM   word1  word2 ... => V
doc1    0      1
doc2    => D(Collection)
O(|Q|*D*|D|)
```

```python
[In:]  DTM
[Out:] defaultdict(<function __main__.<lambda>()>,
            {0: defaultdict(int, {'오류': 1, '우회': 1, '교수': 12, .....}
```

In [6]:
# List 만으로는 표현에 한계가 존재한다
from collections import defaultdict 
DTM = defaultdict(lambda:defaultdict(int))

# 자료형 생성하기
for i, doc in enumerate(collection):
    for term in Mecab().nouns(doc):
        DTM[i][term] += 1

# Token 이 내용의 확인 : {doc1:{}}
print("Collection 문서 수:", len(DTM))
DTM[0]["교수"]  # DTM[문서 인덱스][검색 Token]

Collection 문서 수: 103


12

## **2 TDM 모델 만들기**
- DTM 모델을 활용하여 **Token 별 검색 Tree** 를 재구성 합니다
- **엔트로피가 낮다** : 식별력이 높은 단어 (출현 빈도가 차이가 큰 모델링이 가능)
- **[DTM/ TDM by scikit-learn ](https://medium.com/@omicro03/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC-nlp-7%EC%9D%BC%EC%B0%A8-term-document-matrix-tdm-f959ce229ade)**

```json
TDM    doc1 doc2 ... => D(Collection)
word1    0    1
word2    1
word3    => V
|Q|*D, (Linked List) // DTM 과 비교해 간단 (단점은 미리 작업할 내용이 많아 유연한 운영엔 단점)
word1, ptr => doc2, ptr => doc5, .. // 
```

In [7]:
# TDM : DTM 을 활용한 색인용 Token 사전을 메모리에서 관리 한다
TDM = defaultdict(lambda:defaultdict(int))
for doc, termDict in DTM.items():
    for term, freq in termDict.items():
        TDM[term][doc] = freq

list(TDM.keys())[:12]

['오류', '우회', '함수', '추가', '교수', '점', '만점', '학생', '혁진', '코리아', '텍', '설문']

In [8]:
# 
query        = "김정은 판문점"
searchResult = []

for _ in word_tokenize(query):
    docResult = [] # collection 문서별 Token 검색결과
    for doc, termDict in DTM.items():
        if _ in termDict.keys():
            docResult.append(doc)
    searchResult.append(docResult)

# [ 김정은 검색결과,  판문점 검색결과 ]
searchResult

[[1, 36, 72, 88, 102], [1, 10, 20, 36, 50]]

In [9]:
# InterSection 작업결과 출력 (키워드 중복결과)
# group = searchResult[0]
# for _ in searchResult[1:]:
#     group = set(group).intersection(_)

group = [_ for _ in searchResult[0]  if _ in searchResult[1]]
group

[1, 36]

In [10]:
# TMD 모델의 결과를 DTM 으로 확인하기
# Query Token 이 Document 에 포함여부 확인
[q in DTM[1]   for q in word_tokenize(query) ]

[True, True]

In [11]:
DTM[0].keys()

dict_keys(['오류', '우회', '함수', '추가', '교수', '점', '만점', '학생', '혁진', '코리아', '텍', '설문', '조사', '결과', '연구', '환경', '인식', '차이', '불', '실', '연합뉴스', '자료', '사진', '대전', '이재림', '기자', '국내', '청년', '과학자', '간', '차', '것', '진로', '관심', '여부', '간극', '파악', '일', '한국연구재단', '한국', '산업', '기술', '대학교', '재단', '정책', '혁신', '팀', '이해', '당사자', '심층', '진단', '내용', '이슈', '리포트', '발표', '월', '이공', '대학원', '박사', '후', '연구원', '천', '명', '상대', '분야', '지원', '사업', '수행', '책임자', '연구실', '문화', '수', '문항', '응답', '분석', '항목', '긍정', '경우', '비율', '부정', '압도', '수준', '기준', '표준', '값', '집단', '정도', '적극', '지도', '제자', '이상', '이', '지난해', '국제', '학술지', '네이처', '리더십', '문제', '공개', '외국', '사례', '상담', '동료', '선배', '차별', '과제', '참여', '경제', '보상', '집필진', '외', '비교', '때', '우리', '나라', '과학', '시사', '바', '노력', '필요', '지적', '웹', '네이버', '채널', '구독', '흥'])

In [12]:
"한국연구재단" in DTM[0].keys()

True

## **3 Inversed Index 검색모델 만들기**
[Python 을 활용한 Posting 구조 실습하기](https://medium.com/@fro_g/writing-a-simple-inverted-index-in-python-3c8bcb52169a)
- in-Memory 인덱스 : **Voca Token or Lexicon** (단어 => 어느문서? 위치 포인터)
- **On-Disk :** 문서의 몇번째 정보로 저장 (**Post = on-dick** 데이터로 세분화)
- 즉 **Token** 마다 **검색용 Token Tree** 를 미리 생성 및 활용 **(검색 비용의 최소화** 및 **키워드 관리**)

In [13]:
# 검색 query 내용을 포함하는 문장들 찾기
query = "김정은 판문점"

searchResult = []
for q in word_tokenize(query):
    searchResult.append(list(TDM[q].keys()))
searchResult

[[1, 36, 72, 88, 102], [1, 10, 20, 36, 50]]

In [14]:
# Vocabluary or Lexicon = in-memory 에 저장(단어 => 어느문서? 위치(포인터))
# Posting = on-disk : Huge Data (어느문서?, )
Vocabulary       = [] # TDM 저장된 단어목록
globalVocabulary = {} # in-Memory : 단어와 위치값
globalPosting    = [] # on-Dist : 디스크 문서저장

for i, doc in enumerate(collection):
    localTDM = defaultdict(int) # 문서별 TDM 계산객체
    
    # 문서별 단어의 빈도를 기록한다.
    for term in Mecab().nouns(doc):
        localTDM[term] += 1
        
    for term, freq in localTDM.items():
        if term not in Vocabulary:
            Vocabulary.append(term)
            
        termIdx = Vocabulary.index(term)
        
        if termIdx not in globalVocabulary.keys():
            nextPtr = -1 # 더이상 해당 Token 이 발견되지 않을 때
        else:
            nextPtr = globalVocabulary[termIdx] # 
        
        _posting = [i, freq, nextPtr]        
        globalVocabulary[termIdx] = len(globalPosting)
        globalPosting.append(_posting)

len(globalVocabulary), max(globalVocabulary.keys())

(5431, 5430)

In [15]:
# 검색 Query Token 을 활용한 Tree 내용 살펴보기
query = "판문점"
ptr   = globalVocabulary[Vocabulary.index(query)]
while True:
    if ptr < 0:
        break
    print(globalPosting[ptr])
    ptr = globalPosting[ptr][-1]

[50, 1, 6830]
[36, 3, 3738]
[20, 1, 2097]
[10, 1, 139]
[1, 2, -1]


In [16]:
# TDM 모델을 활용한 Query 검색엔진
# 검색결과를 빠르게 출력 합니다.
query = "김정은 판문점"

searchResult = list()
for q in word_tokenize(query):
    ptr = globalVocabulary[Vocabulary.index(q)]
    _qResult = list()
    while True:
        _posting = globalPosting[ptr]
        _qResult.append(_posting[0])
        if _posting[-1] == -1:
            break
        ptr = _posting[-1]
    searchResult.append(_qResult)

searchResult

[[102, 88, 72, 36, 1], [50, 36, 20, 10, 1]]